# Red Neuronal RNN

In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import scipy 
import numpy as np
import random as rd
import nltk
import pickle
import sklearn
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import datetime
from tqdm import tqdm

from sklearn.metrics import classification_report, roc_auc_score

%matplotlib inline
nltk.download('stopwords')

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\igna-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Leo y spliteo el set de datos

In [2]:
tweets = pd.read_csv('train_EN.csv')

In [3]:
train = tweets.iloc[:,1:3]
train.dropna(inplace =True)

In [4]:
train['tweet'] = train['tweet'].str.lower()
train['tweet'] = train['tweet'].astype(str)

Creo la matriz de embbedings

In [5]:
import gensim
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [6]:
 w2v_model.vectors.shape

(3000000, 300)

In [7]:
emb_matrix = np.concatenate([np.zeros((1,300)), 
                             np.random.normal(size=(1,300)),
                             w2v_model.vectors[:500000]], axis=0)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train['tweet'], train['sarcastic'], test_size=0.2, train_size=0.8, random_state=1999)

In [9]:
X_test = X_test.dropna()
y_test = y_test.dropna()

In [10]:
tokenized_texts_test = []
for text in tqdm(X_test):
    tokenized_texts_test.append(nltk.word_tokenize(text))

100%|██████████████████████████████████████████████████████████████████████████████| 694/694 [00:00<00:00, 1784.05it/s]


In [11]:
tokenized_texts = []
for text in tqdm(X_train):
    tokenized_texts.append(nltk.word_tokenize(text))

100%|████████████████████████████████████████████████████████████████████████████| 2773/2773 [00:00<00:00, 3200.51it/s]


In [12]:
MAX_LEN = int(np.percentile([len(t) for t in tokenized_texts], 95))
MAX_LEN

50

In [13]:
def preprocess_toks(tokenized_text):
    tokenized_text =tokenized_text.copy()
    if len(tokenized_text) > MAX_LEN:
        tokenized_text = tokenized_text[:MAX_LEN]
    for i in range(len(tokenized_text)):
        if tokenized_text[i] in w2v_model and w2v_model.key_to_index[tokenized_text[i]] < 500000:
            tokenized_text[i] = w2v_model.key_to_index[tokenized_text[i]]+2
        else:
            tokenized_text[i] = 1
    return tokenized_text

In [14]:
emb_matrix.shape

(500002, 300)

In [15]:
for i in tqdm(range(len(tokenized_texts)), total=len(tokenized_texts)):
    tokenized_texts[i] = preprocess_toks(tokenized_texts[i])

100%|████████████████████████████████████████████████████████████████████████████| 2773/2773 [00:00<00:00, 4553.36it/s]


In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenized_texts = pad_sequences(
    tokenized_texts, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post', value=0
)

In [17]:
for i in tqdm(range(len(tokenized_texts_test)), total=len(tokenized_texts_test)):
    tokenized_texts_test[i] = preprocess_toks(tokenized_texts_test[i])

100%|██████████████████████████████████████████████████████████████████████████████| 694/694 [00:00<00:00, 2891.74it/s]


In [18]:
tokenized_texts_test = pad_sequences(
    tokenized_texts_test, maxlen=MAX_LEN, dtype='int32', padding='post',
    truncating='post', value=0
)

In [19]:
from keras.layers import Dense, Input, GRU, Embedding, LSTM
from keras.models import Model

In [20]:
from keras.layers import Conv1D, GlobalMaxPooling1D, Dropout, Concatenate, Reshape, Flatten, MaxPooling1D

In [21]:
word_indexes = Input((50,), dtype='int32')

word_emb = Embedding(500002, 300, weights=[emb_matrix], 
                     trainable=False, mask_zero=True)

emb_sequence = word_emb(word_indexes) # (22, 300)
rnn1 = GRU(50, activation='tanh')(emb_sequence) # (1,50)
dense1 = Dense(50, activation='tanh')(rnn1)
dense2 = Dense(50, activation='tanh')(dense1)
dense3 = Dense(50, activation='tanh')(dense2)

out = Dense(1, activation='sigmoid')(dense3)
model = Model(inputs=word_indexes, outputs=out)

In [22]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[keras.metrics.AUC()])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [23]:
history = model.fit(tokenized_texts, y_train.values, validation_split=0.1, callbacks=[callback], epochs=200,batch_size=2777)

Epoch 1/200
1/1 [==============================] - 21s 21s/step - loss: 0.8188 - auc: 0.5008 - val_loss: 0.7684 - val_auc: 0.5356
Epoch 2/200
1/1 [==============================] - 1s 1s/step - loss: 0.7977 - auc: 0.5005 - val_loss: 0.7531 - val_auc: 0.5403
Epoch 3/200
1/1 [==============================] - 1s 1s/step - loss: 0.7772 - auc: 0.5009 - val_loss: 0.7386 - val_auc: 0.5340
Epoch 4/200
1/1 [==============================] - 1s 1s/step - loss: 0.7576 - auc: 0.5009 - val_loss: 0.7250 - val_auc: 0.5322
Epoch 5/200
1/1 [==============================] - 1s 1s/step - loss: 0.7388 - auc: 0.5030 - val_loss: 0.7124 - val_auc: 0.5272
Epoch 6/200
1/1 [==============================] - 1s 1s/step - loss: 0.7210 - auc: 0.5039 - val_loss: 0.7008 - val_auc: 0.5185
Epoch 7/200
1/1 [==============================] - 1s 1s/step - loss: 0.7043 - auc: 0.5059 - val_loss: 0.6903 - val_auc: 0.5011
Epoch 8/200
1/1 [==============================] - 1s 1s/step - loss: 0.6889 - auc: 0.5064 - val_loss:

In [24]:
preds = model.predict(tokenized_texts_test)

In [25]:
y_pred = (preds >= 0.5).astype(bool)

In [26]:
print(classification_report(y_test, y_pred))
roc_auc_score(y_test, preds)

              precision    recall  f1-score   support

           0       0.77      0.99      0.86       532
           1       0.14      0.01      0.01       162

    accuracy                           0.76       694
   macro avg       0.45      0.50      0.44       694
weighted avg       0.62      0.76      0.66       694



0.458727838113803

In [60]:
1 - 0.458727838113803

0.541272161886197

## Creo una nueva red con LSTM

In [27]:
word_indexes = Input((50,), dtype='int32')

word_emb = Embedding(500002, 300, weights=[emb_matrix], 
                     trainable=False, mask_zero=True)

emb_sequence = word_emb(word_indexes)
emb_sequence = Dropout(0.25)(emb_sequence)
rnn1 = LSTM(50, activation='tanh')(emb_sequence)
dense1 = Dense(50, activation='tanh')(rnn1)
dense2 = Dense(50, activation='tanh')(dense1)
dense3 = Dense(50, activation='tanh')(dense2)

out = Dense(1, activation='sigmoid')(dense3)
model = Model(inputs=word_indexes, outputs=out)

In [28]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[keras.metrics.AUC()])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

In [29]:
history = model.fit(tokenized_texts, y_train.values, validation_split=0.1, callbacks=[callback], epochs=100, batch_size=256)

Epoch 1/100
10/10 [==============================] - 23s 1s/step - loss: 0.6911 - auc_1: 0.4935 - val_loss: 0.6556 - val_auc_1: 0.4310
Epoch 2/100
10/10 [==============================] - 3s 284ms/step - loss: 0.6124 - auc_1: 0.5088 - val_loss: 0.6440 - val_auc_1: 0.4337
Epoch 3/100
10/10 [==============================] - 3s 292ms/step - loss: 0.5875 - auc_1: 0.4981 - val_loss: 0.6532 - val_auc_1: 0.4337
Epoch 4/100
10/10 [==============================] - 3s 299ms/step - loss: 0.5795 - auc_1: 0.5033 - val_loss: 0.6550 - val_auc_1: 0.4369
Epoch 5/100
10/10 [==============================] - 3s 287ms/step - loss: 0.5778 - auc_1: 0.4944 - val_loss: 0.6505 - val_auc_1: 0.4377
Epoch 6/100
10/10 [==============================] - 3s 284ms/step - loss: 0.5744 - auc_1: 0.4961 - val_loss: 0.6416 - val_auc_1: 0.4436
Epoch 7/100
10/10 [==============================] - 3s 246ms/step - loss: 0.5695 - auc_1: 0.5161 - val_loss: 0.6347 - val_auc_1: 0.4602
Epoch 8/100
10/10 [========================

In [30]:
len(history.history['loss'])

26

In [31]:
preds = model.predict(tokenized_texts_test)

In [32]:
y_pred = (preds >= 0.5).astype(bool)

In [33]:
print(classification_report(y_test, y_pred))
roc_auc_score(y_test, preds)

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       532
           1       0.00      0.00      0.00       162

    accuracy                           0.77       694
   macro avg       0.38      0.50      0.43       694
weighted avg       0.59      0.77      0.67       694



0.5669845911073981

Con la mejora del EarlyStopping y la LTSM empeoro.


## Pruebo Conv1D

In [34]:
word_indexes = Input((50,), dtype='int32')

word_emb = Embedding(500002, 300, weights=[emb_matrix], 
                     trainable=False, mask_zero=True)

emb_sequence = word_emb(word_indexes)
conv1 = Conv1D(10,1,1,padding = 'same')(emb_sequence)
conv1drop = Dropout(0.10)(conv1)
conv2 = Conv1D(10,2,1,padding = 'same')(emb_sequence)
conv2drop = Dropout(0.10)(conv2)
conv3 = Conv1D(10,3,1,padding = 'same')(emb_sequence)
conv3drop = Dropout(0.10)(conv3)
concatted = Concatenate()([conv1drop, conv2drop, conv3drop])
colap = GlobalMaxPooling1D()(concatted)
dense1 = Dense(30, activation='tanh')(colap)
dense1drop = Dropout(0.10)(dense1)
dense2 = Dense(30, activation='tanh')(dense1drop)
dense2drop = Dropout(0.10)(dense2)
dense3 = Dense(30, activation='tanh')(dense2drop)


out = Dense(1, activation='sigmoid')(dense3)
model = Model(inputs=word_indexes, outputs=out)

In [35]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 50, 300)      150000600   ['input_3[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 50, 10)       3010        ['embedding_2[0][0]']            
                                                                                                  
 conv1d_1 (Conv1D)              (None, 50, 10)       6010        ['embedding_2[0][0]']            
                                                                                            

In [36]:
y_train.values.reshape(-1,1).shape

(2773, 1)

In [37]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[keras.metrics.AUC()])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=12,restore_best_weights=True)

In [38]:
history = model.fit(tokenized_texts, y_train.values, validation_split=0.1, callbacks=[callback], epochs=200,batch_size=64)

Epoch 1/200
39/39 [==============================] - 13s 123ms/step - loss: 0.5679 - auc_2: 0.5074 - val_loss: 0.6133 - val_auc_2: 0.5321
Epoch 2/200
39/39 [==============================] - 2s 58ms/step - loss: 0.5670 - auc_2: 0.5122 - val_loss: 0.6416 - val_auc_2: 0.5601
Epoch 3/200
39/39 [==============================] - 2s 52ms/step - loss: 0.5601 - auc_2: 0.5458 - val_loss: 0.6293 - val_auc_2: 0.5228
Epoch 4/200
39/39 [==============================] - 2s 48ms/step - loss: 0.5630 - auc_2: 0.5223 - val_loss: 0.6245 - val_auc_2: 0.5685
Epoch 5/200
39/39 [==============================] - 2s 61ms/step - loss: 0.5557 - auc_2: 0.5516 - val_loss: 0.6071 - val_auc_2: 0.6057
Epoch 6/200
39/39 [==============================] - 2s 51ms/step - loss: 0.5402 - auc_2: 0.6275 - val_loss: 0.6537 - val_auc_2: 0.5890
Epoch 7/200
39/39 [==============================] - 2s 52ms/step - loss: 0.4897 - auc_2: 0.7409 - val_loss: 0.6310 - val_auc_2: 0.5913
Epoch 8/200
39/39 [===========================

In [39]:
preds = model.predict(tokenized_texts_test)

In [40]:
y_pred = (preds >= 0.5).astype(bool)

In [41]:
print(classification_report(y_test, y_pred))
roc_auc_score(y_test, preds)

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       532
           1       0.00      0.00      0.00       162

    accuracy                           0.77       694
   macro avg       0.38      0.50      0.43       694
weighted avg       0.59      0.77      0.67       694



0.58994708994709

## Pruebo Conv1D combinado con LSTM

In [53]:
word_indexes = Input((50,), dtype='int32')

word_emb = Embedding(500002, 300, weights=[emb_matrix], 
                     trainable=False, mask_zero=True)

emb_sequence = word_emb(word_indexes)
gru = GRU(30, activation='tanh',dropout=0.1)(emb_sequence)
rnn1 = Dropout(0.40)(gru)
conv1 = Conv1D(10,1,1,padding = 'same')(emb_sequence)
conv1drop = Dropout(0.40)(conv1)
conv2 = Conv1D(10,2,1,padding = 'same')(emb_sequence)
conv2drop = Dropout(0.40)(conv2)
conv3 = Conv1D(10,3,1,padding = 'same')(emb_sequence)
conv3drop = Dropout(0.40)(conv3)
concatted = Concatenate()([conv1drop, conv2drop, conv3drop])
colap = GlobalMaxPooling1D()(concatted)
concatted2 = Concatenate()([colap,rnn1])
dense1 = Dense(40, activation='tanh')(concatted2)
dense1drop = Dropout(0.10)(dense1)
dense2 = Dense(40, activation='tanh')(dense1drop)
dense2drop = Dropout(0.10)(dense2)
dense3 = Dense(40, activation='tanh')(dense2drop)


out = Dense(1, activation='sigmoid')(dense3)
model = Model(inputs=word_indexes, outputs=out)

In [54]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 50, 300)      150000600   ['input_5[0][0]']                
                                                                                                  
 conv1d_6 (Conv1D)              (None, 50, 10)       3010        ['embedding_4[0][0]']            
                                                                                                  
 conv1d_7 (Conv1D)              (None, 50, 10)       6010        ['embedding_4[0][0]']            
                                                                                            

In [55]:
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[keras.metrics.AUC()])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=200,restore_best_weights=True)

In [56]:
history = model.fit(tokenized_texts, y_train.values, validation_split=0.1, callbacks=[callback], epochs=200,batch_size=2777)

Epoch 1/200
1/1 [==============================] - 25s 25s/step - loss: 0.6611 - auc_4: 0.5022 - val_loss: 0.8277 - val_auc_4: 0.5372
Epoch 2/200
1/1 [==============================] - 3s 3s/step - loss: 0.6796 - auc_4: 0.5087 - val_loss: 0.6633 - val_auc_4: 0.5431
Epoch 3/200
1/1 [==============================] - 2s 2s/step - loss: 0.5911 - auc_4: 0.4831 - val_loss: 0.6373 - val_auc_4: 0.5588
Epoch 4/200
1/1 [==============================] - 2s 2s/step - loss: 0.6147 - auc_4: 0.5106 - val_loss: 0.6150 - val_auc_4: 0.5383
Epoch 5/200
1/1 [==============================] - 2s 2s/step - loss: 0.5871 - auc_4: 0.5132 - val_loss: 0.6179 - val_auc_4: 0.5188
Epoch 6/200
1/1 [==============================] - 2s 2s/step - loss: 0.5647 - auc_4: 0.5098 - val_loss: 0.6513 - val_auc_4: 0.4854
Epoch 7/200
1/1 [==============================] - 2s 2s/step - loss: 0.5749 - auc_4: 0.5059 - val_loss: 0.6622 - val_auc_4: 0.4998
Epoch 8/200
1/1 [==============================] - 2s 2s/step - loss: 0.58

1/1 [==============================] - 3s 3s/step - loss: 0.5475 - auc_4: 0.5383 - val_loss: 0.6228 - val_auc_4: 0.5013
Epoch 63/200
1/1 [==============================] - 3s 3s/step - loss: 0.5465 - auc_4: 0.5528 - val_loss: 0.6219 - val_auc_4: 0.4739
Epoch 64/200
1/1 [==============================] - 3s 3s/step - loss: 0.5461 - auc_4: 0.5396 - val_loss: 0.6219 - val_auc_4: 0.5295
Epoch 65/200
1/1 [==============================] - 3s 3s/step - loss: 0.5453 - auc_4: 0.5528 - val_loss: 0.6216 - val_auc_4: 0.4808
Epoch 66/200
1/1 [==============================] - 3s 3s/step - loss: 0.5452 - auc_4: 0.5499 - val_loss: 0.6225 - val_auc_4: 0.4787
Epoch 67/200
1/1 [==============================] - 3s 3s/step - loss: 0.5446 - auc_4: 0.5465 - val_loss: 0.6226 - val_auc_4: 0.5050
Epoch 68/200
1/1 [==============================] - 3s 3s/step - loss: 0.5468 - auc_4: 0.5309 - val_loss: 0.6243 - val_auc_4: 0.4959
Epoch 69/200
1/1 [==============================] - 3s 3s/step - loss: 0.5453 - au

1/1 [==============================] - 3s 3s/step - loss: 0.5384 - auc_4: 0.5568 - val_loss: 0.6307 - val_auc_4: 0.5181
Epoch 124/200
1/1 [==============================] - 3s 3s/step - loss: 0.5392 - auc_4: 0.5511 - val_loss: 0.6275 - val_auc_4: 0.5075
Epoch 125/200
1/1 [==============================] - 3s 3s/step - loss: 0.5407 - auc_4: 0.5349 - val_loss: 0.6261 - val_auc_4: 0.4959
Epoch 126/200
1/1 [==============================] - 4s 4s/step - loss: 0.5379 - auc_4: 0.5665 - val_loss: 0.6260 - val_auc_4: 0.5070
Epoch 127/200
1/1 [==============================] - 3s 3s/step - loss: 0.5397 - auc_4: 0.5456 - val_loss: 0.6306 - val_auc_4: 0.5030
Epoch 128/200
1/1 [==============================] - 3s 3s/step - loss: 0.5377 - auc_4: 0.5770 - val_loss: 0.6403 - val_auc_4: 0.4908
Epoch 129/200
1/1 [==============================] - 3s 3s/step - loss: 0.5400 - auc_4: 0.5288 - val_loss: 0.6467 - val_auc_4: 0.5002
Epoch 130/200
1/1 [==============================] - 3s 3s/step - loss: 0.53

Epoch 184/200
1/1 [==============================] - 3s 3s/step - loss: 0.4666 - auc_4: 0.8204 - val_loss: 0.8144 - val_auc_4: 0.5265
Epoch 185/200
1/1 [==============================] - 3s 3s/step - loss: 0.4288 - auc_4: 0.8321 - val_loss: 0.7638 - val_auc_4: 0.5245
Epoch 186/200
1/1 [==============================] - 3s 3s/step - loss: 0.4399 - auc_4: 0.8209 - val_loss: 0.7636 - val_auc_4: 0.5272
Epoch 187/200
1/1 [==============================] - 3s 3s/step - loss: 0.4371 - auc_4: 0.8185 - val_loss: 0.7970 - val_auc_4: 0.5381
Epoch 188/200
1/1 [==============================] - 3s 3s/step - loss: 0.4159 - auc_4: 0.8362 - val_loss: 0.8912 - val_auc_4: 0.5375
Epoch 189/200
1/1 [==============================] - 3s 3s/step - loss: 0.4256 - auc_4: 0.8403 - val_loss: 0.8910 - val_auc_4: 0.5425
Epoch 190/200
1/1 [==============================] - 3s 3s/step - loss: 0.4045 - auc_4: 0.8478 - val_loss: 0.8616 - val_auc_4: 0.5385
Epoch 191/200
1/1 [==============================] - 3s 3s/ste

In [57]:
preds = model.predict(tokenized_texts_test)

In [58]:
y_pred = (preds >= 0.5).astype(bool)

In [59]:
print(classification_report(y_test, y_pred))
roc_auc_score(y_test, preds)

              precision    recall  f1-score   support

           0       0.78      0.90      0.83       532
           1       0.32      0.15      0.20       162

    accuracy                           0.73       694
   macro avg       0.55      0.53      0.52       694
weighted avg       0.67      0.73      0.69       694



0.6121437853893995